#### Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pickle 


#### Loading Dataset and Preprocessing

In [ ]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

In [ ]:
# Dropping Unnecessary Columns
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1)
df.head()

In [ ]:
print("Number of Columns: ", df.columns)

In [ ]:
# Handling Gender Column
encoder_gender = LabelEncoder()
df['Gender'] = encoder_gender.fit_transform(df['Gender'])
df

In [ ]:
# Handling Geography Column
encoder_geo = OneHotEncoder()
encoder_geo_array = encoder_geo.fit_transform(df[['Geography']]).toarray()
encoder_geo_array


In [ ]:
encoder_geo.get_feature_names_out(['Geography'])

In [ ]:
df

In [ ]:
geo_df = pd.DataFrame(encoder_geo_array, columns=encoder_geo.get_feature_names_out(['Geography']))
geo_df

In [ ]:
df = pd.concat([df.drop('Geography', axis = 1), geo_df], axis = 1)
df

In [ ]:
# Spliting X & y
X = df.drop('Exited', axis = 1)
y = df['Exited']

#### Spliting data into Training and Test Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
X_train

#### Standard Scaling

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Saving StandardScaler, OneHotEncoder, LabelEncoder Pickle file

In [ ]:
with open('encoder_gender.pkl', 'wb') as file:
    pickle.dump(encoder_gender, file)

with open('encoder_geo.pkl', 'wb') as file:
    pickle.dump(encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
X_train

### ANN Implementation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

In [ ]:
(X_train.shape[1],)

In [ ]:
model = Sequential([
    Input(shape = (X_train.shape[1], )),
    Dense(64, activation = 'relu'), # DL1 
    Dense(32, activation = 'relu'), # DL2
    Dense(1, activation = 'sigmoid') # Output layer
])

In [ ]:
model.summary()

In [ ]:
# fixing optimiser & loss with learning rate
opt = tf.keras.optimizers.Adam(learning_rate = 0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
# compile without learning rate
# model.compile(
#     optimiser = 'adam',
#     loss = 'binary_crossentropy',
#     metrics = ['accuracy']
# )


# compile the model
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy']
)

In [ ]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [ ]:
# Training the model
history = model.fit(
    X_train, y_train,
    epochs = 100,
    validation_split = 0.2,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

In [ ]:
# saving the model
model.save("model.keras")


In [ ]:
## Load Tensorboard Extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'loss: {loss: .4f}')
print(f'Accuracy: {accuracy: .4f}')